## Litics360 Prototype Version Pv1.0
# Partitioning and Splitting Voter Registration Database (OHIO)

### Purpose:
#1. Extract voters with declared PARTY_AFFLIIATION
#2. Partition declared voters dataset into two -- Blind data for final testing and data for building model
#3. Split MODEL data for building into two sets -- Pre and post
#4. Split PRE for building into two sets -- Cleaning and for Training
#5. Split POST into two sets -- Cross Validation and for testing
#6. Export separately.

--------------------------------------------------------------------------------------------------------------------
# Library Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as split

--------------------------------------------------------------------------------------------------------------------
# Data Import

In [2]:
#Import file1
voterdata_file1 = pd.read_csv(
    "data/VoterData/sourced/SWVF_1_22.txt", 
    sep=",", quotechar='"', header=0, encoding='ISO-8859-1', na_values=['NA'],low_memory=False)

In [3]:
#Import file2
voterdata_file2 = pd.read_csv(
    "data/VoterData/sourced/SWVF_23_44.txt", 
    sep=",", quotechar='"', header=0, encoding='ISO-8859-1', na_values=['NA'],low_memory=False)

In [4]:
#Import file3
voterdata_file3 = pd.read_csv(
    "data/VoterData/sourced/SWVF_45_66.txt", 
    sep=",", quotechar='"', header=0, encoding='ISO-8859-1', na_values=['NA'],low_memory=False)

In [5]:
#Import file4
voterdata_file4 = pd.read_csv(
    "data/VoterData/sourced/SWVF_67_88.txt", 
    sep=",", quotechar='"', header=0, encoding='ISO-8859-1', na_values=['NA'],low_memory=False)

In [6]:
#Combine all into one dataframe
voterdata_df = pd.concat([voterdata_file1,voterdata_file2,voterdata_file3,voterdata_file4], 
                         axis=0, join='outer', join_axes=None, ignore_index=False,
                         keys=None, levels=None, names=None, verify_integrity=False, copy=True)

--------------------------------------------------------------------------------------------------------------------
# #1: Extract Voters with Declared Party Affliation

In [8]:
#Number of voters
print("# of Registered Voters: ", len(voterdata_df))

#Unique types of party affiliations listed in dataset
parties = voterdata_df.PARTY_AFFILIATION.unique()
print("\nParty Affiliations Listed Types: ", parties)
print("--> nan = Undeclared\n--> D = Democrat\n--> R = Republican\n--> G = Green\n--> L = Libertarian")

#Number of undeclared/declared voters
num_undeclaredvoters = sum(pd.isna(voterdata_df['PARTY_AFFILIATION']))
print("\n# of Undeclared Party Affiliation: ", num_undeclaredvoters)

num_declaredvoters = len(voterdata_df)-(sum(pd.isna(voterdata_df['PARTY_AFFILIATION'])))
print("# of Declared Party Affiliation: ", num_declaredvoters)

# of Registered Voters:  7700810

Party Affiliations Listed Types:  [nan 'D' 'R' 'G' 'L']
--> nan = Undeclared
--> D = Democrat
--> R = Republican
--> G = Green
--> L = Libertarian

# of Undeclared Party Affiliation:  4459592
# of Declared Party Affiliation:  3241218


In [11]:
#Extract all voters with declared party into dataset
df = voterdata_df[pd.notnull(voterdata_df['PARTY_AFFILIATION'])]

#Check number of rows -- match  to above
print("Match #declared-voters to #rows of df:\n", num_declaredvoters,"<-->",len(df))

Match #declared-voters to #rows of df:
 3241218 <--> 3241218


--------------------------------------------------------------------------------------------------------------------
# #2. Partition Dataset: Blind Data and Model Data
1. BLIND - 10%
2. MODEL - 90%

In [12]:
#Partition into blind and model - 10/90, random_state none, shuffle True, 
model_df, blind_df= split(
    df, train_size=0.9, test_size=0.1, random_state=None, shuffle=True)

#Total declared voter count
print("Total declared voters: ", len(df))

#Calculate percentage, check # of rows in BLIND df
print("\n10% of", len(df) ,"=",round(0.1*3241218),"\n BLIND dataset =",len(blind_df))

#Calculate percentage, check # of rows in MODEL df
print("\n90% of", len(df) ,"=",round(0.9*3241218),"\n MODEL dataset =",len(model_df))

Total declared voters:  3241218

10% of 3241218 = 324122 
 BLIND dataset = 324122

90% of 3241218 = 2917096 
 MODEL dataset = 2917096


--------------------------------------------------------------------------------------------------------------------
# #3. Split Model Dataset: Prep and Test
1. PRE - 60%
3. POST - 40%

In [13]:
#Split MODEL dataset into prep and test - 60/40, shuffle, random state none
pre_df, post_df= split(
    model_df, test_size=0.4, train_size=0.6, random_state=None, shuffle=True)

#Total declared voter count
print("Total in Model dataset: ", len(model_df))

#Calculate percentage, check # of rows in PREP df
print("\n60% of", len(model_df) ,"=",round(0.6*2917096),"\n PRE dataset =",len(pre_df))

#Calculate percentage, check # of rows in TEST df
print("\n40% of", len(model_df) ,"=",round(0.4*2917096),"\n POST dataset =",len(post_df))

Total in Model dataset:  2917096

60% of 2917096 = 1750258 
 PRE dataset = 1750257

40% of 2917096 = 1166838 
 POST dataset = 1166839


--------------------------------------------------------------------------------------------------------------------
# #4. Split Pre Dataset: Exploration and Training
1. EXPLORE_CLEAN - 5% 
2. TRAIN - 95%

In [28]:
#Split PRE dataset into explore and train - 5/95, shuffle, random state none
train_df, explore_clean_df= split(
    pre_df, test_size=0.05, train_size=0.95, random_state=None, shuffle=True)

#Total declared voter count
print("Total in PRE dataset: ", len(pre_df))

#Calculate percentage, check # of rows in EXPLORE df
print("\n5% of", len(pre_df) ,"=",round(0.05*1750257),"\n EXPLORE_CLEAN dataset =",len(explore_clean_df))

#Calculate percentage, check # of rows in TRAIN df
print("\n95% of", len(pre_df) ,"=",round(0.95*1750257),"\n TRAIN dataset =",len(train_df))

Total in PRE dataset:  1750257

5% of 1750257 = 87513 
 EXPLORE_CLEAN dataset = 87513

95% of 1750257 = 1662744 
 TRAIN dataset = 1662744


--------------------------------------------------------------------------------------------------------------------
# #5. Split Post Dataset: Cross-Validation and Testing
1. VALIDATE - 50% 
2. TEST - 50%

In [15]:
#Split POST dataset into explore and train - 50/50, shuffle, random state none
validate_df, test_df= split(
    post_df, test_size=0.5, train_size=0.5, random_state=None, shuffle=True)

#Total declared voter count
print("Total in POST dataset: ", len(post_df))

#Calculate percentage, check # of rows in VALIDATE df
print("\n50% of", len(post_df) ,"=",round(0.5*1166839),"\n VALIDATE dataset =",len(validate_df))

#Calculate percentage, check # of rows in TEST df
print("\n50% of", len(post_df) ,"=",round(0.5*1166839),"\n TEST dataset =",len(test_df))

Total in POST dataset:  1166839

50% of 1166839 = 583420 
 VALIDATE dataset = 583419

50% of 1166839 = 583420 
 TEST dataset = 583420


--------------------------------------------------------------------------------------------------------------------
# #6. Export 5 datasets -- Blind, explore/clean, train, validate, test

### Research and Explore Data
1. explore_clean_df --> data/VoterData/split/explore_clean_RAW_voterdata.csv

### Train/Cross-Validation
2. train_df --> data/VoterData/split/train_RAW_voterdata.csv
3. validate_df --> data/VoterData/split/validate_RAW_voterdata.csv

### Test
4. test_df --> data/VoterData/split/test_RAW_voterdata.csv
5. blind_df --> data/VoterData/blind/blind_RAW_voterdata.csv

In [19]:
#Function for easy export to csv
def df2csv(df,filename):
    path = 'data/VoterData/'+filename+'_RAW_voterdata.csv'
    df.to_csv(path, encoding='utf-8', mode='a', header=True, index=False)

In [21]:
#Run function to export
df2csv(explore_clean_df, 'explore_clean/explore_clean')
df2csv(train_df, 'train/train')
df2csv(test_df, 'valid_test/test')
df2csv(validate_df, 'valid_test/validate')
df2csv(blind_df, 'blind/blind')